source http://darques.eu/blog/index.php/2017/07/27/mlbox-a-short-regression_tutorial/

In [1]:
#Import MLBox and other packages
import mlbox as mlb #I don't really like * imports

ModuleNotFoundError: No module named 'mlbox'

In [ ]:
# Read files using preprocessing.reader
# Usage: train_test_split(path to training data,path to test data,target)
# Target is "SalePrice", i.e. the price the houses are sold
data = mlb.preprocessing.Reader(sep = ",").train_test_split(["data.csv","test.csv"], 'SalePrice')

In [ ]:
data

In [ ]:
# Preprocess data
# 1/ Remove Ids
# 2/ Delete drifting data between train and test sets
data = mlb.preprocessing.Drift_thresholder().fit_transform(data)

In [ ]:
space_ligtgbm = {
                'ne__numerical_strategy'    :{"search":"choice",
                                              "space":[0,'mean','median','most_frequent']},
                'ne__categorical_strategy'  :{"search":"choice",
                                              "space":[np.NaN,"None"]},
                'ce__strategy'              :{"search":"choice",
                                              "space":['label_encoding','entity_embedding','dummification']},
                'fs__strategy'              :{"search":"choice",
                                              "space":['l1','variance','rf_feature_importance']},
                'fs__threshold'             :{"search":"uniform",
                                              "space":[0.01,0.6]},
                'est__strategy'             :{"search":"choice",
                                              "space":["LightGBM"]},
                'est__max_depth'            :{"search":"choice",
                                              "space":[3,4,5,6,7]},
                'est__learning_rate'        :{"search":"uniform",
                                              "space":[0.01,0.1]},
                'est__subsample'            :{"search":"uniform",
                                              "space":[0.4,0.9]},
                'est__reg_alpha'            :{"search":"uniform",
                                              "space":[0,10]},
                'est__reg_lambda'           :{"search":"uniform",
                                              "space":[0,10]},
                'est__n_estimators'         :{"search":"choice",
                                              "space":[1000,1250,1500]}
                }

In [ ]:
# Optimisation
best_lightgbm = mlb.optimisation.Optimiser(scoring = "r2",n_folds = 5).optimise(space_ligtgbm, data, 120)

In [ ]:
best_lightgbm

In [ ]:
# Predict on the test data using best parameters
mlb.prediction.Predictor().fit_predict(best_lightgbm, data)

In [ ]:
# Train and evaluate with default parameters
# best is a set of parameters that were estimated as being the best
# default = mlb.optimisation.Optimiser().evaluate(None, data)